In [5]:

from photutils.aperture import aperture_photometry, SkyCircularAperture
from astropy.table import Table

import photutils

from astropy.io import fits

from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import ascii

In [7]:
my_vandercat_complete = ascii.read('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv').to_pandas()

# 1) Weight Maps:

In [26]:
# Load the weight maps for each band
weight_maps = {
   
    'f125w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_wht.fits'),
    'f140w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_wht.fits'),
    'f160w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_wht.fits'),
    'f275w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/UV candels/hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_wht.fits'),
    'f435w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_wht.fits'),
    'f606w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_wht.fits'),
    'f775w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_wht.fits'),
    'f850w': fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_wht.fits'),
   
}

In [34]:
# Load your clump catalog
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv'
clump_catalog = ascii.read(clump_catalog_path).to_pandas()

# Define aperture radius here
aperture_radius = 0.18  #in arcseconds

# Define the band suffix to WCS file mapping 
band_to_wcs_file = {
    '125': 'goodss_all_wfc3_ir_f125w_060mas_v1.0_wht.fits',
    '140': 'goodss_all_wfc3_ir_f140w_060mas_v1.0_wht.fits',
    '160': 'goodss_all_wfc3_ir_f160w_060mas_v1.0_wht.fits',
    '275': 'hlsp_uvcandels_hst_wfc3-uvis_goodss-60mas_f275w_v1.0_wht.fits',
    '435': 'goodss_all_acs_wfc_f435w_060mas_v1.5_wht.fits',
    '606': 'goodss_all_acs_wfc_f606w_060mas_v1.5_wht.fits',
    '775': 'goodss_all_acs_wfc_f775w_060mas_v1.5_wht.fits',
    '850': 'goodss_all_acs_wfc_f850l_060mas_v1.5_wht.fits',
    
      
}

bands = ['275','435', '606', '775', '850', '125', '140', '160']
# Calculate uncertainty
def calculate_uncertainty(clump_row, wcs, weight_map, aperture_radius):

    # Convert RA and DEC to pixel coordinates
    x, y = wcs.all_world2pix(clump_row['RA_clump'], clump_row['DEC_clump'], 0)
    #  Create a SkyCoord object representing the center of the aperture
    aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
    
    aperture = SkyCircularAperture(aperture_center, r=aperture_radius* u.arcsec)
    photometry_data = aperture_photometry(weight_map, aperture , wcs=wcs)
    sum_weights = np.sum(photometry_data['aperture_sum'])

    # Calculate uncertainty
    if sum_weights == 0:
        uncertainty = np.nan  # Handle divide by zero
    else:
        uncertainty = np.sqrt(1 / sum_weights)
    
    return uncertainty
    # uncertainty = np.sqrt(1 / sum_weights)
    # return uncertainty

# Initialize a dictionary to store the uncertainties for each band
uncertainties = {band: [] for band in bands}

# Loop over each clump in catalog
for index, clump_row in clump_catalog.iterrows():
    if clump_row['DEC_clump'] == -99.0:
        # If invalid coordinates, append NaN for all bands and continue to the next iteration
        for band_suffix in bands:
            uncertainties[band_suffix].append(np.nan)
        print(f"Skipping clump due to invalid coordinates: {clump_row['CANDELS_ID']}, {clump_row['Clump_ID']}")
        continue
    else:
        # Create SkyCoord object from clump coordinates
        clump_coords = SkyCoord(ra=clump_row['RA_clump']*u.deg, dec=clump_row['DEC_clump']*u.deg)
    
    for band_suffix in bands:
        # Get the weight map for the current band
        weight_map = fits.getdata('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/' + band_to_wcs_file[band_suffix])
        
        # Extract the WCS for the current band from the weight map FITS file
        with fits.open('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/' + band_to_wcs_file[band_suffix]) as hdul:
            wcs = WCS(hdul[0].header)
            wcs.sip = None  # Disable any SIP distortion if present
        
        # Calculate the uncertainty for this clump and band
        uncertainty = calculate_uncertainty(clump_row, wcs, weight_map, aperture_radius)
        
        # Store the calculated uncertainty
        uncertainties[band_suffix].append(uncertainty)

# Add the uncertainties to the clump catalog DataFrame
for band_suffix in bands:
  

    clump_catalog[f'Uncertainty_{band_suffix}_mjy'] = uncertainties[band_suffix]

# Save the updated catalog with uncertainties
clump_catalog.to_csv('/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/uncertainty__included_clump_catalog_my_vander.csv', index=False)

Skipping clump due to invalid coordinates: 135, -99
Skipping clump due to invalid coordinates: 137, -99
Skipping clump due to invalid coordinates: 145, -99
Skipping clump due to invalid coordinates: 544, -99
Skipping clump due to invalid coordinates: 583, -99
Skipping clump due to invalid coordinates: 606, -99
Skipping clump due to invalid coordinates: 675, -99
Skipping clump due to invalid coordinates: 733, -99
Skipping clump due to invalid coordinates: 835, -99
Skipping clump due to invalid coordinates: 879, -99
Skipping clump due to invalid coordinates: 946, -99
Skipping clump due to invalid coordinates: 986, -99
Skipping clump due to invalid coordinates: 1132, -99
Skipping clump due to invalid coordinates: 1183, -99
Skipping clump due to invalid coordinates: 1187, -99
Skipping clump due to invalid coordinates: 1198, -99
Skipping clump due to invalid coordinates: 1203, -99
Skipping clump due to invalid coordinates: 1208, -99
Skipping clump due to invalid coordinates: 1485, -99
Skipp

# Poisson noise:

In [35]:
band_to_science_file = {
    '125': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_drz.fits',
    '140': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_drz.fits',
    '160': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits',
    '435': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits',
    '606': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_drz.fits',
    '775': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_drz.fits',
    '850': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_drz.fits',


}

In [36]:
band_to_exptime_file = {
    '125': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_exp.fits',
    '140': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_exp.fits',
    '160': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_exp.fits',
    '435': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_exp.fits',
    '606': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_exp.fits',
    '775': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_exp.fits',
    '850': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_exp.fits',

}

In [2]:
band_to_files = {
    '125': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f125w_060mas_v1.0_exp.fits'
    },
    '140': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f140w_060mas_v1.0_exp.fits'
    },
    '160': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_wfc3_ir_f160w_060mas_v1.0_exp.fits'
    },
    '435': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f435w_060mas_v1.5_exp.fits'
    },
    '606': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f606w_060mas_v1.5_exp.fits'
    },
    '775': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f775w_060mas_v1.5_exp.fits'
    },
    '850': {
        'science': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_drz.fits',
        'exptime': '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/CANDELS-data/mosaics/goodss_all_acs_wfc_f850l_060mas_v1.5_exp.fits'
    },
}

In [2]:
# #  Function to calculate uncertainty
# def calculate_poisson_noise(clump_row, wcs, science_image, exptime_image, aperture_radius):
#     # Convert RA and DEC to pixel coordinates
#     x, y = wcs.all_world2pix(clump_row['RA_clump'], clump_row['DEC_clump'], 0)
#     # Create a SkyCoord object representing the center of the aperture
#     aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
#     aperture = SkyCircularAperture(aperture_center, r=aperture_radius * u.arcsec)
    
#     # Calculate counts (science image x exposure time image)
#     counts_image = science_image * exptime_image
    
#     # Perform aperture photometry to sum the counts within the aperture
#     photometry_data = aperture_photometry(counts_image, aperture, wcs=wcs)
#     counts_sum = photometry_data['aperture_sum'][0]
    
#     # Calculate Poisson noise as the square root of the counts sum
#     poisson_noise = np.sqrt(counts_sum) / np.mean(exptime_image)
    
#     return poisson_noise


In [41]:
# Initialize a dictionary to store the Poisson uncertainties for each band
poisson_uncertainties = {band: [] for band in band_to_files.keys()}

In [42]:

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.table import Table
import pandas as pd
import numpy as np
from photutils import SkyCircularAperture, aperture_photometry
import astropy.units as u


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/658513016.py:7: DeprecationWarning: `photutils.SkyCircularAperture` is a deprecated alias for `photutils.aperture.SkyCircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import SkyCircularAperture` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/658513016.py:7: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry


In [4]:
bands = ['435', '606', '775', '850', '125', '140', '160']

In [6]:
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv'
clump_catalog = Table.read(clump_catalog_path).to_pandas()

In [45]:
# Define aperture radius 
aperture_radius = 0.18  # in arcseconds

In [46]:
#  Pre-load FITS data into memory
fits_data = {}
for band, paths in band_to_files.items():
    with fits.open(paths['science']) as hdul_science:
        wcs = WCS(hdul_science[0].header, naxis=2)
        science_image = hdul_science[0].data

    with fits.open(paths['exptime']) as hdul_exptime:
        exptime_image = hdul_exptime[0].data

    fits_data[band] = {'wcs': wcs, 'science_image': science_image, 'exptime_image': exptime_image}


In [47]:
def calculate_poisson_noise(clump_row, band, fits_data, aperture_radius):
    # Extract necessary data from the pre-loaded FITS data dictionary
    wcs = fits_data[band]['wcs']
    science_image = fits_data[band]['science_image']
    exptime_image = fits_data[band]['exptime_image']
 


    # Step 1: Create a counts image (science image multiplied by exposure time image)
    counts_image = science_image * exptime_image

    # Step 2: Calculate the square root of the counts image
    sqrt_counts_image = np.sqrt(counts_image)
  

    # Step 3: Normalize the square root by dividing by the exposure time
    normalized_sqrt_image = sqrt_counts_image / exptime_image

    # Step 4: Calculate the square root of the normalized image
    final_image = np.square(normalized_sqrt_image)

    # Step 5: Perform aperture photometry on the final image
    aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
    aperture = SkyCircularAperture(aperture_center, r=aperture_radius * u.arcsec).to_pixel(wcs)
    photometry_data = aperture_photometry(final_image, aperture)
    poisson_noise = np.sqrt(photometry_data['aperture_sum'][0])

    return poisson_noise



In [48]:

# Calculate Poisson uncertainty for the first clump
first_clump_row = clump_catalog.iloc[0]
poisson_uncertainties = {}
for band in band_to_files.keys():
    poisson_uncertainty = calculate_poisson_noise(first_clump_row, band,fits_data, aperture_radius)
    poisson_uncertainties[band] = poisson_uncertainty
    print(f'Band {band}: Poisson Uncertainty = {poisson_uncertainty}')

# # Add the Poisson uncertainties to the DataFrame
# for band in band_to_files.keys():
#     clump_catalog.at[0, f'Poisson_Uncertainty_{band}'] = poisson_uncertainties[band]

# # Save the updated first row with Poisson uncertainties
# output_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/first_clump_poisson_uncertainty.csv'
# clump_catalog.iloc[0:1].to_csv(output_path, index=False)

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:13: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:17: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Band 125: Poisson Uncertainty = 1.9932890341093104


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:13: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:17: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Band 140: Poisson Uncertainty = nan


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:13: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:17: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


...

Band 606: Poisson Uncertainty = nan


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:13: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:17: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Band 775: Poisson Uncertainty = nan


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:13: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_10178/852731129.py:17: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Band 850: Poisson Uncertainty = nan


In [8]:
first_clump_row = clump_catalog.iloc[0]
print(first_clump_row)

Unnamed: 0            0
Field            goodss
CANDELS_ID           18
z                 2.658
RA_gal        53.103527
                ...    
GOODS                 0
recno             71882
GroupID               1
GroupSize             2
Separation     0.012801
Name: 0, Length: 61, dtype: object


# getting Nan values ,editting that:

In [7]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from photutils import SkyCircularAperture, aperture_photometry
import astropy.units as u

# Define aperture radius
aperture_radius = 0.18  # in arcseconds

# Pre-load FITS data into memory
fits_data = {}
for band, paths in band_to_files.items():
    with fits.open(paths['science']) as hdul_science:
        wcs = WCS(hdul_science[0].header, naxis=2)
        science_image = hdul_science[0].data
    with fits.open(paths['exptime']) as hdul_exptime:
        exptime_image = hdul_exptime[0].data
        # Replace zeros in the exposure time image to prevent division by zero
        exptime_image[exptime_image <= 0] = np.nan  # Use NaN where exposure time is zero

    fits_data[band] = {'wcs': wcs, 'science_image': science_image, 'exptime_image': exptime_image}

def calculate_poisson_noise(clump_row, band, fits_data, aperture_radius):
    wcs = fits_data[band]['wcs']
    science_image = fits_data[band]['science_image']
    exptime_image = fits_data[band]['exptime_image']

    # Handle any infinities or NaNs before operations
    np.nan_to_num(science_image, copy=False, nan=0.0, posinf=0.0, neginf=0.0)

    # Create a counts image
    counts_image = science_image * exptime_image

    # Safely calculate the square root of the counts image
    sqrt_counts_image = np.sqrt(np.maximum(counts_image, 0))

    # Safely normalize the square root by dividing by the exposure time
    with np.errstate(divide='ignore', invalid='ignore'):
        normalized_sqrt_image = sqrt_counts_image / exptime_image
        normalized_sqrt_image[~np.isfinite(normalized_sqrt_image)] = 0  # Set inf or NaN to 0

    # Calculate the square root of the normalized image
    final_image = np.sqrt(np.maximum(normalized_sqrt_image, 0))

    # Perform aperture photometry
    aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
    aperture = SkyCircularAperture(aperture_center, r=aperture_radius * u.arcsec).to_pixel(wcs)
    photometry_data = aperture_photometry(final_image, aperture)
    poisson_noise = photometry_data['aperture_sum'][0] if not np.isnan(photometry_data['aperture_sum'][0]) else np.nan

    return poisson_noise

# Calculate Poisson uncertainty for the first clump
first_clump_row = clump_catalog.iloc[0]
poisson_uncertainties = {}
for band in band_to_files.keys():
    poisson_uncertainty = calculate_poisson_noise(first_clump_row, band, fits_data, aperture_radius)
    poisson_uncertainties[band] = poisson_uncertainty
    print(f'Band {band}: Poisson Uncertainty = {poisson_uncertainty}')

# # Add the Poisson uncertainties to the DataFrame
# for band in fits_data.keys():
#     clump_catalog.at[0, f'Poisson_Uncertainty_{band}'] = poisson_uncertainties[band]

# # Save the updated first row with Poisson uncertainties
# output_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/first_clump_poisson_uncertainty.csv'
# clump_catalog.iloc[0:1].to_csv(output_path, index=False)


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_31082/2240592904.py:5: DeprecationWarning: `photutils.SkyCircularAperture` is a deprecated alias for `photutils.aperture.SkyCircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import SkyCircularAperture` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_31082/2240592904.py:5: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry


Band 125: Poisson Uncertainty = 1.9932890341093104
Band 140: Poisson Uncertainty = 0.0
Band 160: Poisson Uncertainty = 2.2373757848768876
Band 435: Poisson Uncertainty = 0.7176286216566575
Band 606: Poisson Uncertainty = 1.4254808636175598
Band 775: Poisson Uncertainty = 1.3659354070445804
Band 850: Poisson Uncertainty = 1.105387018984923


# Final edit:

In [1]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.table import Table
import pandas as pd
import numpy as np
from photutils import SkyCircularAperture, aperture_photometry
import astropy.units as u

# Paths to the cutouts 
cutout_files_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies/'

# Load the clump catalog
clump_catalog_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/last_aperturecorrcted_vander.csv'
clump_catalog = Table.read(clump_catalog_path).to_pandas()

# Define aperture radius 
aperture_radius = 0.18  # in arcseconds

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3895529235.py:7: DeprecationWarning: `photutils.SkyCircularAperture` is a deprecated alias for `photutils.aperture.SkyCircularAperture` and will be removed in the future. Instead, please use `from photutils.aperture import SkyCircularAperture` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3895529235.py:7: DeprecationWarning: `photutils.aperture_photometry` is a deprecated alias for `photutils.aperture.aperture_photometry` and will be removed in the future. Instead, please use `from photutils.aperture import aperture_photometry` to silence this warning.
  from photutils import SkyCircularAperture, aperture_photometry


In [13]:


example_fits_file = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies/goodss_18_f775w_images.fits'

# Open the FITS file to inspect its structure
with fits.open(example_fits_file) as hdul:
    hdul.info()


Filename: /Users/neal/Documents/PhD/projects/SED-fitting/Clump/Cutouts_galaxies/goodss_18_f775w_images.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      28   (200, 200)   float64   
  1                1 ImageHDU         7   (200, 200)   float32   
  2                1 ImageHDU         7   (200, 200)   float32   
  3                1 ImageHDU         5   ()      


In [23]:
# Function to calculate Poisson noise
def calculate_poisson_noise(clump_row, band, aperture_radius):
    # Construct the path to the cutout FITS file
    if band == '850':
        # Use 'lp' instead of 'w' for the 850 band
        cutout_file =f"{cutout_files_path}goodss_{clump_row['ID']}_f850lp_images.fits"
    else:
        cutout_file = f"{cutout_files_path}goodss_{clump_row['ID']}_f{band}w_images.fits"
    
    # Open the FITS file and extract data
    with fits.open(cutout_file) as hdul:
        wcs = WCS(hdul[0].header, naxis=2) # Assuming the WCS is in the primary HDU header
        science_image = hdul[0].data  # The science image 
        exptime_image = hdul[2].data  # Assuming the exposure time image is the third extension 


        # Perform calculations as before
        counts_image = science_image * exptime_image
        # Replace non-positive values in counts_image with a small positive number
    # to prevent sqrt from encountering invalid values
        counts_image[counts_image <= 0] = np.finfo(float).eps
        sqrt_counts_image = np.sqrt(counts_image)
        normalized_sqrt_image = sqrt_counts_image / exptime_image

        # Replace any remaining non-positive values in exptime_image to avoid division by zero
        non_positive_mask = exptime_image <= 0
        normalized_sqrt_image[non_positive_mask] = np.finfo(float).eps


        final_image = np.square(normalized_sqrt_image)

        # Perform aperture photometry
        aperture_center = SkyCoord(clump_row['RA_clump'], clump_row['DEC_clump'], unit='deg')
        aperture = SkyCircularAperture(aperture_center, r=aperture_radius * u.arcsec).to_pixel(wcs)
        photometry_data = aperture_photometry(final_image, aperture)
        poisson_noise = np.sqrt(photometry_data['aperture_sum'][0])

    return poisson_noise


In [21]:
# Calculate Poisson uncertainty for each clump and band
for i, clump_row in clump_catalog.iterrows():
    poisson_uncertainties = {}
    for band in ['435', '606', '775', '850', '125', '140', '160']:
        poisson_uncertainty = calculate_poisson_noise(clump_row, band, aperture_radius)
        poisson_uncertainties[band] = poisson_uncertainty
        print(f'Clump ID {clump_row["ID"]} Band {band}: Poisson Uncertainty = {poisson_uncertainty}')


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:

Clump ID 18 Band 435: Poisson Uncertainty = 0.005559460162114893
Clump ID 18 Band 606: Poisson Uncertainty = 0.016210789229255503
Clump ID 18 Band 775: Poisson Uncertainty = nan
Clump ID 18 Band 850: Poisson Uncertainty = 0.008648984684297905
Clump ID 18 Band 125: Poisson Uncertainty = 0.02923253956992375
Clump ID 18 Band 140: Poisson Uncertainty = nan
Clump ID 18 Band 160: Poisson Uncertainty = 0.035915393228941245
Clump ID 18 Band 435: Poisson Uncertainty = 0.0038223092440637287
Clump ID 18 Band 606: Poisson Uncertainty = 0.011993226177163327
Clump ID 18 Band 775: Poisson Uncertainty = 0.012009387512666798
Clump ID 18 Band 850: Poisson Uncertainty = 0.005457865248220632
Clump ID 18 Band 125: Poisson Uncertainty = 0.023442848161352375
Clump ID 18 Band 140: Poisson Uncertainty = nan
Clump ID 18 Band 160: Poisson Uncertainty = nan
Clump ID 31 Band 435: Poisson Uncertainty = 0.0074695129308711295
Clump ID 31 Band 606: Poisson Uncertainty = 0.017399173582050776
Clump ID 31 Band 775: Poiss

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:

Clump ID 40 Band 160: Poisson Uncertainty = 0.024558767963969053
Clump ID 40 Band 435: Poisson Uncertainty = 0.009094507492430105
Clump ID 40 Band 606: Poisson Uncertainty = 0.021775991275897202
Clump ID 40 Band 775: Poisson Uncertainty = 0.021516330877045797
Clump ID 40 Band 850: Poisson Uncertainty = 0.01372626754786144
Clump ID 40 Band 125: Poisson Uncertainty = 0.05380760140984597
Clump ID 40 Band 140: Poisson Uncertainty = nan
Clump ID 40 Band 160: Poisson Uncertainty = 0.048770587984521975
Clump ID 40 Band 435: Poisson Uncertainty = 0.009670444763105197
Clump ID 40 Band 606: Poisson Uncertainty = 0.028999456064939762
Clump ID 40 Band 775: Poisson Uncertainty = 0.025372728020249098
Clump ID 40 Band 850: Poisson Uncertainty = 0.012570192701686028
Clump ID 40 Band 125: Poisson Uncertainty = 0.053722265377726126
Clump ID 40 Band 140: Poisson Uncertainty = nan
Clump ID 40 Band 160: Poisson Uncertainty = 0.05071743604948874
Clump ID 40 Band 435: Poisson Uncertainty = 0.0067290158673472

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value en

Clump ID 79 Band 775: Poisson Uncertainty = 0.012555094266782972
Clump ID 79 Band 850: Poisson Uncertainty = 0.0068327210017403404
Clump ID 79 Band 125: Poisson Uncertainty = 0.039266029920003324
Clump ID 79 Band 140: Poisson Uncertainty = nan
Clump ID 79 Band 160: Poisson Uncertainty = 0.035654756046723336
Clump ID 79 Band 435: Poisson Uncertainty = 0.006594364576040103
Clump ID 79 Band 606: Poisson Uncertainty = 0.015820994082518414
Clump ID 79 Band 775: Poisson Uncertainty = 0.01640301020764783
Clump ID 79 Band 850: Poisson Uncertainty = 0.009349749983818625
Clump ID 79 Band 125: Poisson Uncertainty = 0.04930702106822529
Clump ID 79 Band 140: Poisson Uncertainty = nan
Clump ID 79 Band 160: Poisson Uncertainty = 0.04965270783138275
Clump ID 79 Band 435: Poisson Uncertainty = 0.00687916105656031
Clump ID 79 Band 606: Poisson Uncertainty = 0.015311755394076563
Clump ID 79 Band 775: Poisson Uncertainty = 0.014881665101788007
Clump ID 79 Band 850: Poisson Uncertainty = 0.0062323978726470

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning

Clump ID 79 Band 606: Poisson Uncertainty = nan
Clump ID 79 Band 775: Poisson Uncertainty = nan
Clump ID 79 Band 850: Poisson Uncertainty = nan
Clump ID 79 Band 125: Poisson Uncertainty = nan
Clump ID 79 Band 140: Poisson Uncertainty = nan
Clump ID 79 Band 160: Poisson Uncertainty = nan
Clump ID 107 Band 435: Poisson Uncertainty = 0.012574946941468922
Clump ID 107 Band 606: Poisson Uncertainty = 0.03460839929618693
Clump ID 107 Band 775: Poisson Uncertainty = nan
Clump ID 107 Band 850: Poisson Uncertainty = 0.01736055094324218
Clump ID 107 Band 125: Poisson Uncertainty = 0.045352184576813545
Clump ID 107 Band 140: Poisson Uncertainty = nan
Clump ID 107 Band 160: Poisson Uncertainty = 0.04119696903161842
Clump ID 107 Band 435: Poisson Uncertainty = 0.0137350309636914
Clump ID 107 Band 606: Poisson Uncertainty = 0.03156362037962288
Clump ID 107 Band 775: Poisson Uncertainty = nan
Clump ID 107 Band 850: Poisson Uncertainty = 0.015446130028894037
Clump ID 107 Band 125: Poisson Uncertainty 

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:20: RuntimeWarning: invalid value encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning: invalid value encountered in sqrt
  sqrt_counts_image = np.sqrt(counts_image)
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3067684843.py:19: RuntimeWarning

ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got -99.0 deg

In [24]:
def is_valid_coord(ra, dec):
    try:
        SkyCoord(ra, dec, unit='deg')
        return True
    except ValueError:
        return False

# Add empty columns for uncertainties in the DataFrame
for band in ['435', '606', '775', '850', '125', '140', '160']:
    clump_catalog[f'Poisson_Uncertainty_{band}'] = np.nan

# Calculate Poisson uncertainty for each clump and band
for i, clump_row in clump_catalog.iterrows():
    # Check for valid coordinates before proceeding
    if not is_valid_coord(clump_row['RA_clump'], clump_row['DEC_clump']):
        print(f"Skipping clump ID {clump_row['ID']} due to invalid coordinates.")
        continue

    for band in ['435', '606', '775', '850', '125', '140', '160']:
        try:
            poisson_uncertainty = calculate_poisson_noise(clump_row, band, aperture_radius)
        except ValueError as e:
            print(f"Skipping clump ID {clump_row['ID']} Band {band} due to error: {e}")
            continue
        
        # Check for NaN values before appending
        if not np.isnan(poisson_uncertainty):
            clump_catalog.at[i, f'Poisson_Uncertainty_{band}'] = poisson_uncertainty
            print(f'Clump ID {clump_row["ID"]} Band {band}: Poisson Uncertainty = {poisson_uncertainty}')
        else:
            print(f'Clump ID {clump_row["ID"]} Band {band}: Poisson Uncertainty is NaN')

# Now you can save the updated DataFrame with the uncertainties
output_path = '/Users/neal/Documents/PhD/projects/SED-fitting/Clump/results/poisson_uncertainty_included.csv'
clump_catalog.to_csv(output_path, index=False)


/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj

Clump ID 18 Band 435: Poisson Uncertainty = 0.005559460162114893
Clump ID 18 Band 606: Poisson Uncertainty = 0.016210789229255503
Clump ID 18 Band 775: Poisson Uncertainty = 0.033148674525862655
Clump ID 18 Band 850: Poisson Uncertainty = 0.008648984684297905
Clump ID 18 Band 125: Poisson Uncertainty = 0.02923253956992375
Clump ID 18 Band 140: Poisson Uncertainty = 1.1806994923525508e-15
Clump ID 18 Band 160: Poisson Uncertainty = 0.035915393228941245
Clump ID 18 Band 435: Poisson Uncertainty = 0.0038223092440637287
Clump ID 18 Band 606: Poisson Uncertainty = 0.011993226177163327
Clump ID 18 Band 775: Poisson Uncertainty = 0.012009387512666798
Clump ID 18 Band 850: Poisson Uncertainty = 0.005457865248220632
Clump ID 18 Band 125: Poisson Uncertainty = 0.023442848161352375
Clump ID 18 Band 140: Poisson Uncertainty = 1.180699479876795e-15
Clump ID 18 Band 160: Poisson Uncertainty = 0.030468347895868438
Clump ID 31 Band 435: Poisson Uncertainty = 0.0074695129308711295
Clump ID 31 Band 606:

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj

Clump ID 40 Band 775: Poisson Uncertainty = 0.021516330877045797
Clump ID 40 Band 850: Poisson Uncertainty = 0.01372626754786144
Clump ID 40 Band 125: Poisson Uncertainty = 0.05380760140984597
Clump ID 40 Band 140: Poisson Uncertainty = 1.1806993599080854e-15
Clump ID 40 Band 160: Poisson Uncertainty = 0.048770587984521975
Clump ID 40 Band 435: Poisson Uncertainty = 0.009670444763105197
Clump ID 40 Band 606: Poisson Uncertainty = 0.028999456064939762
Clump ID 40 Band 775: Poisson Uncertainty = 0.025372728020249098
Clump ID 40 Band 850: Poisson Uncertainty = 0.012570192701686028
Clump ID 40 Band 125: Poisson Uncertainty = 0.053722265377726126
Clump ID 40 Band 140: Poisson Uncertainty = 1.1806993596878228e-15
Clump ID 40 Band 160: Poisson Uncertainty = 0.05071743604948874
Clump ID 40 Band 435: Poisson Uncertainty = 0.0067290158673472805
Clump ID 40 Band 606: Poisson Uncertainty = 0.01865270964218137
Clump ID 40 Band 775: Poisson Uncertainty = 0.014484502188398939
Clump ID 40 Band 850: Po

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


...

Clump ID 26464 Band 775: Poisson Uncertainty = 0.0041600143603043
Clump ID 26464 Band 850: Poisson Uncertainty = 0.0016499389955605855
Clump ID 26464 Band 125: Poisson Uncertainty = 0.008845354687950911
Clump ID 26464 Band 140: Poisson Uncertainty = 0.02882200115111491
Clump ID 26464 Band 160: Poisson Uncertainty = 0.006864730606744588
Clump ID 26464 Band 435: Poisson Uncertainty = 0.00401365978939404
Clump ID 26464 Band 606: Poisson Uncertainty = 0.005790847669869892
Clump ID 26464 Band 775: Poisson Uncertainty = 0.004213186099191876
Clump ID 26464 Band 850: Poisson Uncertainty = 0.0023217352495042694
Clump ID 26464 Band 125: Poisson Uncertainty = 0.01005706789897986
Clump ID 26464 Band 140: Poisson Uncertainty = 0.032518090421340955
Clump ID 26464 Band 160: Poisson Uncertainty = 0.008758689810371027
Clump ID 26464 Band 435: Poisson Uncertainty = 0.003953714362868875
Clump ID 26464 Band 606: Poisson Uncertainty = 0.007114722379172225
Clump ID 26464 Band 775: Poisson Uncertainty = 0.00

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image
/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Clump ID 26531 Band 125: Poisson Uncertainty = 0.008092488229912322
Clump ID 26531 Band 140: Poisson Uncertainty = 0.019079279062035194
Clump ID 26531 Band 160: Poisson Uncertainty = 0.008283212680985428
Clump ID 26531 Band 435: Poisson Uncertainty = 0.0037608461243985414
Clump ID 26531 Band 606: Poisson Uncertainty = 0.0064129475897867876
Clump ID 26531 Band 775: Poisson Uncertainty = 0.004473356776648162
Clump ID 26531 Band 850: Poisson Uncertainty = 0.002155527891586331
Clump ID 26531 Band 125: Poisson Uncertainty = 0.015487361094206722
Clump ID 26531 Band 140: Poisson Uncertainty = 0.046467404155377236
Clump ID 26531 Band 160: Poisson Uncertainty = 0.016483411210241748
Clump ID 26552 Band 435: Poisson Uncertainty = 0.010429633704037811
Clump ID 26552 Band 606: Poisson Uncertainty = 0.017178469571991425
Clump ID 26552 Band 775: Poisson Uncertainty = 0.015390743183925332
Clump ID 26552 Band 850: Poisson Uncertainty = 0.008258477716808597
Clump ID 26552 Band 125: Poisson Uncertainty =

/var/folders/by/26kj85kd3w94p3xv_m2206b40000gn/T/ipykernel_69974/3138385142.py:23: RuntimeWarning: divide by zero encountered in divide
  normalized_sqrt_image = sqrt_counts_image / exptime_image


Clump ID 26732 Band 125: Poisson Uncertainty = 0.017063409442600666
Clump ID 26732 Band 140: Poisson Uncertainty = 0.05098618599903093
Clump ID 26732 Band 160: Poisson Uncertainty = 0.01757409347292779
Clump ID 26732 Band 435: Poisson Uncertainty = 0.006321415603332358
Clump ID 26732 Band 606: Poisson Uncertainty = 0.011683715048246751
Clump ID 26732 Band 775: Poisson Uncertainty = 0.008419959895541857
Clump ID 26732 Band 850: Poisson Uncertainty = 0.0036852910229835537
Clump ID 26732 Band 125: Poisson Uncertainty = 0.01580586625508328
Clump ID 26732 Band 140: Poisson Uncertainty = 0.04186366132175341
Clump ID 26732 Band 160: Poisson Uncertainty = 0.012686503027095153
Clump ID 26732 Band 435: Poisson Uncertainty = 0.004607801751612273
Clump ID 26732 Band 606: Poisson Uncertainty = 0.00858366450042924
Clump ID 26732 Band 775: Poisson Uncertainty = 0.0063506360894798036
Clump ID 26732 Band 850: Poisson Uncertainty = 0.0028710887820624135
Clump ID 26732 Band 125: Poisson Uncertainty = 0.0